In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import os
import pickle
import time
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.models import Model, Sequential
from keras.optimizers import Adadelta, RMSprop, SGD, Adam
from keras.layers import (
    Input,
    Conv1D,
    GlobalAveragePooling1D,
    MaxPooling1D,
    Flatten,
    Activation,
    UpSampling1D,
    AveragePooling1D,
    Reshape,
)
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy, binary_crossentropy


import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

Using TensorFlow backend.


In [2]:
# Import data and normalize
def normalizer(a, min_val, max_val):
    # min-max scaling if wanted
    # https://codereview.stackexchange.com/questions/185785/scale-numpy-array-to-certain-range
    col, row = np.shape(a)
    for i in range(col):
        a[i] = np.interp(a[i], (min_val, max_val), (0, 1))
    return a


# Load Pickles

# all_train
pickle_in = open("all_train_mean_NOSCALE.pickle", "rb")
X_all = pickle.load(pickle_in)

# df_extra1
pickle_in = open("df_extra1_mean_NOSCALE.pickle", "rb")
X_extra1 = pickle.load(pickle_in)

# df_extra2
pickle_in = open("df_extra2_mean_NOSCALE.pickle", "rb")
X_extra2 = pickle.load(pickle_in)

# df_extra3
pickle_in = open("df_extra3_mean_NOSCALE.pickle", "rb")
X_extra3 = pickle.load(pickle_in)

# X_train
pickle_in = open("X_train.pickle", "rb")
X_train = pickle.load(pickle_in)

# X_val
pickle_in = open("X_val.pickle", "rb")
X_val = pickle.load(pickle_in)

# X_test
pickle_in = open("X_test.pickle", "rb")
X_test = pickle.load(pickle_in)

# y_train
pickle_in = open("y_train.pickle", "rb")
y_train = pickle.load(pickle_in)

# y_val
pickle_in = open("y_val.pickle", "rb")
y_val = pickle.load(pickle_in)

# y_test
pickle_in = open("y_test.pickle", "rb")
y_test = pickle.load(pickle_in)

X_all = np.concatenate((X_all, X_extra1, X_extra2, X_extra3), axis=0)

X_kaggle = np.concatenate((X_extra1, X_extra2, X_extra3), axis=0)
                 
# If needed, normalize the data between 0-1
min_all = X_all.min()
max_all = X_all.max()

X_all = normalizer(X_all, min_all, max_all)
X_train = normalizer(X_train, min_all, max_all)
X_val = normalizer(X_val, min_all, max_all)
X_test = normalizer(X_test, min_all, max_all)
X_kaggle = normalizer(X_kaggle, min_all, max_all)

# Reshape
X_all = X_all.reshape([-1, 800, 1]).astype("float32")
X_train = X_train.reshape([-1, 800, 1]).astype("float32")
X_val = X_val.reshape([-1, 800, 1]).astype("float32")
X_test = X_test.reshape([-1, 800, 1]).astype("float32")
X_kaggle = X_kaggle.reshape([-1, 800, 1]).astype("float32")
                 
# split X_all data
X_all_train, X_all_val, y_junk, y_junk = train_test_split(
    X_all, X_all, test_size=0.2, random_state=42
)

**Primary Model: Predict on Test Set for MEAN**

In [3]:
model_name = '/home/tim/Documents/PD-Predictions/notebooks/fc_final/FC_mean_GLOBAL_FALSE-327501p-3e_kern-64e_filt-16pool_size-3d_kern-64d_filt-20fc_units-2fc_layers-0.2dropout_val-1554569587'
model_save = 'epoch.236-val_loss.0.50.h5'
path = os.path.join('/home/tim/Documents/PD-Predictions/notebooks/fc_final/',model_name)


file_path = os.path.join(path,model_save)

model = keras.models.load_model(file_path)

In [4]:
score = model.evaluate(X_val, y_val)
score

157/157 [==============================] - 1s 7ms/step


[0.5025882834841491, 0.7452229348717222]

In [5]:
score = model.evaluate(X_test, y_test)
score

158/158 [==============================] - 0s 832us/step


[0.6046233516705187, 0.7468354392655289]

**Baseline Model (simple CNN): Predict on Test Set**

In [10]:
model_name = '/home/tim/Documents/PD-Predictions/notebooks/cnn_simple/CNN-1235-5kern-30filt-1conv_layer-3kernel2-10filter2-16pool_size-12fc_units-1fc_layers-0dropout_val-1553914440'
model_save = 'epoch.6869-val_loss.0.45.h5'
path = os.path.join('/home/tim/Documents/PD-Predictions/notebooks/cnn_simple',model_name)


file_path = os.path.join(path,model_save)

model = keras.models.load_model(file_path)

In [11]:
score = model.evaluate(X_val, y_val)
score

157/157 [==============================] - 0s 521us/step


[0.44926111789266016, 0.8025477756360534]

In [12]:
score = model.evaluate(X_test, y_test)
score

158/158 [==============================] - 0s 65us/step


[0.6241879285890845, 0.7531645531895794]